In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk

# Use this instead
df = pd.read_csv('us-counties-2020.csv')
df = df.drop(columns=["fips"])
df = df[df["date"] == "2020-12-31"]
df = df.drop(columns=["date"])
display(df)

# df.to_csv('new.csv', index=False)

pop = pd.read_excel('us-counties-pop-2020.xlsx')
pop = pop.dropna()
pop = pop.iloc[1:]  # Drop the US Column
pop = pop.set_axis(['combined', 'April', '2020', '2021'],
                   axis=1, inplace=False)
sus = pop['combined'].str.split(',', expand=True)

sus[0] = sus[0].str[1:-7]
sus[1] = sus[1].str[1:]
# display(sus)

pop = pop.drop(columns=["April", "2021", "combined"])
pop["county"] = sus[0]
pop["state"] = sus[1]
# display(pop)

# display(df)
final = df.merge(pop, how="left", on=["county", "state"])
badData = final[final.isna().any(axis=1)]
# badData.to_csv('bad.csv', index=False)

display(final)
# final.to_csv('complete.csv', index=False)


In [45]:
gdp_raw = pd.read_excel('gdp.xlsx').set_axis(
    ['county', '2017', '2018', '2019', '2020', 'rank 2018', 'percent change 2018', 'percent change 2019', 'percent change 2020', 'rank 2020'], axis=1, inplace=False)
gdp_clean: pd.DataFrame = gdp_raw.drop(columns=['rank 2018', '2017', '2018', '2020',
                                                'rank 2020', 'percent change 2018', 'percent change 2019'], inplace=False).iloc[5:3222]
print(gdp_clean)


          county       2019 percent change 2020
5        Alabama  203383898                -3.2
6        Autauga    1540762                -1.3
7        Baldwin    7134734                -2.1
8        Barbour     729105                -5.8
9           Bibb     380453                 2.1
...          ...        ...                 ...
3217  Sweetwater    3677972               -10.9
3218       Teton    2268742                -4.1
3219       Uinta     881052               -10.3
3220    Washakie     349686                -3.7
3221      Weston     322576                -3.1

[3217 rows x 3 columns]


In [38]:
state_names = pd.read_csv('us-counties-2020.csv')['state'].unique()
print(state_names)
print(len(state_names))


['Washington' 'Illinois' 'California' 'Arizona' 'Massachusetts'
 'Wisconsin' 'Texas' 'Nebraska' 'Utah' 'Oregon' 'Florida' 'New York'
 'Rhode Island' 'Georgia' 'New Hampshire' 'North Carolina' 'New Jersey'
 'Colorado' 'Maryland' 'Nevada' 'Tennessee' 'Hawaii' 'Indiana' 'Kentucky'
 'Minnesota' 'Oklahoma' 'Pennsylvania' 'South Carolina'
 'District of Columbia' 'Kansas' 'Missouri' 'Vermont' 'Virginia'
 'Connecticut' 'Iowa' 'Louisiana' 'Ohio' 'Michigan' 'South Dakota'
 'Arkansas' 'Delaware' 'Mississippi' 'New Mexico' 'North Dakota' 'Wyoming'
 'Alaska' 'Maine' 'Alabama' 'Idaho' 'Montana' 'Puerto Rico'
 'Virgin Islands' 'Guam' 'West Virginia' 'Northern Mariana Islands']
55


In [52]:
counties_df = pd.read_csv('complete.csv')
counties_df['2019 raw GDP'] = pd.NaT  # iloc =5
counties_df['percent change 2020'] = pd.NaT  # iloc = 6

curr_state = None
for index, row in gdp_clean.iterrows():
    if row[0] in state_names:
        curr_state = row[0]
        continue
    else:
        row_index = counties_df.index[(counties_df['state'] == curr_state) & (
            counties_df['county'] == row[0])].tolist()
        counties_df.iloc[[row_index], [5]] = row[1]  # type: ignore
        counties_df.iloc[[row_index], [6]] = row[2]  # type: ignore


In [53]:
print(counties_df)

          county    state  cases  deaths  2020 population 2019 raw GDP  \
0        Autauga  Alabama   4190    48.0          58877.0      1540762   
1        Baldwin  Alabama  13601   161.0         233140.0      7134734   
2        Barbour  Alabama   1514    32.0          25180.0       729105   
3           Bibb  Alabama   1834    46.0          22223.0       380453   
4         Blount  Alabama   4641    63.0          59081.0       932215   
...          ...      ...    ...     ...              ...          ...   
3240  Sweetwater  Wyoming   2966    16.0          42158.0      3677972   
3241       Teton  Wyoming   2138     4.0          23347.0      2268742   
3242       Uinta  Wyoming   1558     7.0          20441.0       881052   
3243    Washakie  Wyoming    780    19.0           7658.0       349686   
3244      Weston  Wyoming    476     2.0           6809.0       322576   

     percent change 2020  
0                   -1.3  
1                   -2.1  
2                   -5.8  
3  